# Understanding Why People Vote

<img src="images/vote.jpg"/>

In August 2006 three researchers (Alan Gerber and Donald Green of Yale University, and Christopher Larimer of the University of Northern Iowa) carried out a large scale field experiment in Michigan, USA to test the hypothesis that one of the reasons people vote is social, or extrinsic, pressure. To quote the first paragraph of their 2008 research paper:

    "Among the most striking features of a democratic political system is the participation of millions of voters in elections. Why do large numbers of people vote, despite the fact that ... "the casting of a single vote is of no significance where there is a multitude of electors"? One hypothesis is adherence to social norms. Voting is widely regarded as a citizen duty, and citizens worry that others will think less of them if they fail to participate in elections. Voters' sense of civic duty has long been a leading explanation of vote turnout..."

In this homework problem we will use both logistic regression and classification trees to analyze the data they collected.

### The Data

The researchers grouped about 344,000 voters into different groups randomly - about 191,000 voters were a "control" group, and the rest were categorized into one of four "treatment" groups. These five groups correspond to five binary variables in the dataset.

    "Civic Duty" (variable civicduty) group members were sent a letter that simply said "DO YOUR CIVIC DUTY - VOTE!"

    "Hawthorne Effect" (variable hawthorne) group members were sent a letter that had the "Civic Duty" message plus the additional message "YOU ARE BEING STUDIED" and they were informed that their voting behavior would be examined by means of public records.
    
    "Self" (variable self) group members received the "Civic Duty" message as well as the recent voting record of everyone in that household and a message stating that another message would be sent after the election with updated records.
    
    "Neighbors" (variable neighbors) group members were given the same message as that for the "Self" group, except the message not only had the household voting records but also that of neighbors - maximizing social pressure.
    
    "Control" (variable control) group members were not sent anything, and represented the typical voting situation.

Additional variables include sex (0 for male, 1 for female), yob (year of birth), and the dependent variable voting (1 if they voted, 0 otherwise).

### Load and Exploration the Data

In [4]:
gerber = read.csv("data/gerber.csv")
head(gerber)

,sex,yob,voting,hawthorne,civicduty,neighbors,self,control
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,1941,0,0,1,0,0,0
2,1,1947,0,0,1,0,0,0
3,1,1982,1,1,0,0,0,0
4,1,1950,1,1,0,0,0,0
5,0,1951,1,1,0,0,0,0
6,1,1959,1,0,0,0,0,1


In [5]:
str(gerber)

'data.frame':	344084 obs. of  8 variables:
 $ sex      : int  0 1 1 1 0 1 0 0 1 0 ...
 $ yob      : int  1941 1947 1982 1950 1951 1959 1956 1981 1968 1967 ...
 $ voting   : int  0 0 1 1 1 1 1 0 0 0 ...
 $ hawthorne: int  0 0 1 1 1 0 0 0 0 0 ...
 $ civicduty: int  1 1 0 0 0 0 0 0 0 0 ...
 $ neighbors: int  0 0 0 0 0 0 0 0 0 0 ...
 $ self     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ control  : int  0 0 0 0 0 1 1 1 1 1 ...


In [6]:
summary(gerber)

      sex              yob           voting         hawthorne    
 Min.   :0.0000   Min.   :1900   Min.   :0.0000   Min.   :0.000  
 1st Qu.:0.0000   1st Qu.:1947   1st Qu.:0.0000   1st Qu.:0.000  
 Median :0.0000   Median :1956   Median :0.0000   Median :0.000  
 Mean   :0.4993   Mean   :1956   Mean   :0.3159   Mean   :0.111  
 3rd Qu.:1.0000   3rd Qu.:1965   3rd Qu.:1.0000   3rd Qu.:0.000  
 Max.   :1.0000   Max.   :1986   Max.   :1.0000   Max.   :1.000  
   civicduty        neighbors          self           control      
 Min.   :0.0000   Min.   :0.000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.0000   Median :0.000   Median :0.0000   Median :1.0000  
 Mean   :0.1111   Mean   :0.111   Mean   :0.1111   Mean   :0.5558  
 3rd Qu.:0.0000   3rd Qu.:0.000   3rd Qu.:0.0000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :1.000   Max.   :1.0000   Max.   :1.0000  

### Problem 1.1 - Exploration and Logistic Regression
We will first get familiar with the data. Load the CSV file gerber.csv into R. What proportion of people in this dataset voted in this election?

In [7]:
z = table(gerber$voting)
z


     0      1 
235388 108696 

In [13]:
# Compute proportion
p = z[2]/sum(z)
paste("Proportion that voted (1): ", round(p,2))

[1] "Proportion that voted (1):  0.32"

### Problem 1.2 - Exploration and Logistic Regression
Which of the four "treatment groups" had the largest percentage of people who actually voted (voting = 1)?

In [16]:
#prop.table(table(data$voting))

prop.table(table(subset(gerber, civicduty==1)$voting))
prop.table(table(subset(gerber, hawthorne==1)$voting))
prop.table(table(subset(gerber, self==1)$voting))
prop.table(table(subset(gerber, neighbors==1)$voting))


        0         1 
0.6854623 0.3145377 


        0         1 
0.6776254 0.3223746 


        0         1 
0.6548485 0.3451515 


        0         1 
0.6220518 0.3779482 

Answer: The neighbors group had the highest population of people voting

### Problem 1.3 - Exploration and Logistic Regression
Build a logistic regression model for voting using the four treatment group variables as the independent variables (civicduty, hawthorne, self, and neighbors). Use all the data to build the model (DO NOT split the data into a training set and testing set). Which of the following coefficients are significant in the logistic regression model?

In [17]:
# Logistic Regression
LogModel = glm(voting ~ civicduty + hawthorne + self + neighbors, data=gerber, family="binomial")

# Summary
summary(LogModel)


Call:
glm(formula = voting ~ civicduty + hawthorne + self + neighbors, 
    family = "binomial", data = gerber)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9744  -0.8691  -0.8389   1.4586   1.5590  

Coefficients:
             Estimate Std. Error  z value Pr(>|z|)    
(Intercept) -0.863358   0.005006 -172.459  < 2e-16 ***
civicduty    0.084368   0.012100    6.972 3.12e-12 ***
hawthorne    0.120477   0.012037   10.009  < 2e-16 ***
self         0.222937   0.011867   18.786  < 2e-16 ***
neighbors    0.365092   0.011679   31.260  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 429238  on 344083  degrees of freedom
Residual deviance: 428090  on 344079  degrees of freedom
AIC: 428100

Number of Fisher Scoring iterations: 4


Answer: Civic Duty, Hawthorne Effect, Self, and Neighbors are significant.

### Problem 1.4 - Exploration and Logistic Regression
Using a threshold of 0.3, what is the accuracy of the logistic regression model? (When making predictions, you don't need to use the newdata argument since we didn't split our data.)

In [28]:
# Make predictions
predictLog = predict(LogModel, type="response")

# Confusion Matrix
zc = table(gerber$voting, predictLog > 0.3)
zc

# Accuracy
accur = sum(diag(zc))/sum(zc)  # (TN+TP)/(TN+FN+TP+FP)
paste("Accuracy Logistic Regression (threshold 0.3): ", round(accur,digits=4))

   
     FALSE   TRUE
  0 134513 100875
  1  56730  51966

[1] "Accuracy Logistic Regression(threshold 0.3):  0.542"

### Problem 1.5 - Exploration and Logistic Regression
Using a threshold of 0.5, what is the accuracy of the logistic regression model?

In [29]:
# Confusion Matrix
zc = table(gerber$voting, predictLog > 0.5)
zc

# Accuracy
accur = sum(diag(zc))/sum(zc)  # (TN+TP)/(TN+FN+TP+FP)
paste("Accuracy Logistic Regression (threshold 0.5): ", round(accur,digits=4))

   
     FALSE
  0 235388
  1 108696

[1] "Accuracy Logistic Regression(threshold 0.5):  0.6841"

### Problem 1.6 - Exploration and Logistic Regression
Compare your previous two answers to the percentage of people who did not vote (the baseline accuracy) and compute the AUC of the model. What is happening here?

In [31]:
library(ROCR)

In [32]:
ROCRpred = prediction(predictLog, gerber$voting)

auc = as.numeric(performance(ROCRpred, "auc")@y.values)

paste("AUC: ", round(auc,digits=4))

[1] "AUC:  0.5308"

Even though all of our variables are significant, our model does not improve over the baseline model of just predicting that someone will not vote, and the AUC is low. So while the treatment groups do make a difference, this is a weak predictive model.

### Problem 2.1 - Trees
We will now try out trees. Build a CART tree for voting using all data and the same four treatment variables we used before. Don't set the option method="class" - we are actually going to create a regression tree here. We are interested in building a tree to explore the fraction of people who vote, or the probability of voting. We’d like CART to split our groups if they have different probabilities of voting. If we used method=‘class’, CART would only split if one of the groups had a probability of voting above 50% and the other had a probability of voting less than 50% (since the predicted outcomes would be different). However, with regression trees, CART will split even if both groups have probability less than 50%.

Leave all the parameters at their default values. You can use the following command in R to build the tree:

CARTmodel = rpart(voting ~ civicduty + hawthorne + self + neighbors, data=gerber)

In [ ]:

Plot the tree. What happens, and if relevant, why?

In [ ]:

Problem 2.2 - Trees
Now build the tree using the command:

CARTmodel2 = rpart(voting ~ civicduty + hawthorne + self + neighbors, data=gerber, cp=0.0) to force the complete tree to be built. Then plot the tree. What do you observe about the order of the splits?

In [ ]:

Problem 2.3 - Trees
Using only the CART tree plot, determine what fraction (a number between 0 and 1) of "Civic Duty" people voted:

In [ ]:

Problem 2.4 - Trees
Make a new tree that includes the "sex" variable, again with cp = 0.0. Notice that sex appears as a split that is of secondary importance to the treatment group.

In the control group, which gender is more likely to vote?

In [ ]:

In the "Civic Duty" group, which gender is more likely to vote?

In [ ]:

Problem 3.1 - Interaction Terms
We know trees can handle "nonlinear" relationships, e.g. "in the 'Civic Duty' group and female", but as we will see in the next few questions, it is possible to do the same for logistic regression. First, let's explore what trees can tell us some more.

Let's just focus on the "Control" treatment group. Create a regression tree using just the "control" variable, then create another tree with the "control" and "sex" variables, both with cp=0.0.

In the "control" only tree, what is the absolute value of the difference in the predicted probability of voting between being in the control group versus being in a different group? You can use the absolute value function to get answer, i.e. abs(Control Prediction - Non-Control Prediction). Add the argument "digits = 6" to the prp command to get a more accurate estimate.

In [ ]:

Problem 3.2 - Interaction Terms
Now, using the second tree (with control and sex), determine who is affected more by NOT being in the control group (being in any of the four treatment groups):

In [ ]:

Problem 3.3 - Interaction Terms
Going back to logistic regression now, create a model using "sex" and "control". Interpret the coefficient for "sex":

Problem 3.4 - Interaction Terms
The regression tree calculated the percentage voting exactly for every one of the four possibilities (Man, Not Control), (Man, Control), (Woman, Not Control), (Woman, Control). Logistic regression has attempted to do the same, although it wasn't able to do as well because it can't consider exactly the joint possibility of being a women and in the control group.

We can quantify this precisely. Create the following dataframe (this contains all of the possible values of sex and control), and evaluate your logistic regression using the predict function (where "LogModelSex" is the name of your logistic regression model that uses both control and sex):

Possibilities = data.frame(sex=c(0,0,1,1),control=c(0,1,0,1)) predict(LogModelSex, newdata=Possibilities, type="response") The four values in the results correspond to the four possibilities in the order they are stated above ( (Man, Not Control), (Man, Control), (Woman, Not Control), (Woman, Control) ). What is the absolute difference between the tree and the logistic regression for the (Woman, Control) case? Give an answer with five numbers after the decimal point.

In [ ]:

Problem 3.5 - Interaction Terms
So the difference is not too big for this dataset, but it is there. We're going to add a new term to our logistic regression now, that is the combination of the "sex" and "control" variables - so if this new variable is 1, that means the person is a woman AND in the control group. We can do that with the following command:

LogModel2 = glm(voting ~ sex + control + sex:control, data=gerber, family="binomial") How do you interpret the coefficient for the new variable in isolation? That is, how does it relate to the dependent variable?

Problem 3.6 - Interaction Terms
Run the same code as before to calculate the average for each group:

predict(LogModel2, newdata=Possibilities, type="response")

Now what is the difference between the logistic regression model and the CART model for the (Woman, Control) case? Again, give your answer with five numbers after the decimal point.

In [ ]:

Problem 3.7 - Interaction Terms
This example has shown that trees can capture nonlinear relationships that logistic regression can not, but that we can get around this sometimes by using variables that are the combination of two variables. Should we always include all possible interaction terms of the independent variables when building a logistic regression model?